In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

pd.set_option('display.float_format', '{:.2f}'.format)

from sklearn.model_selection import train_test_split as tts   # el alias es cosa
from sklearn.ensemble import RandomForestRegressor as rfr
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.preprocessing import StandardScaler
pd.options.mode.copy_on_write = True

In [3]:
df = pd.read_csv("../NBA_csv/StatsAlltrain2.csv")

df['PER'] = (df['PTS'] + (df['AST'] + df['TRB'] + df['STL'] + df['BLK'] - df['TOV']) / df['G']) / (df['MP'] / 48)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2237 entries, 0 to 2236
Data columns (total 31 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Player   2237 non-null   object 
 1   Salario  2237 non-null   int64  
 2   Pos      2237 non-null   object 
 3   Age      2237 non-null   int64  
 4   Tm       2237 non-null   object 
 5   G        2237 non-null   int64  
 6   GS       2237 non-null   int64  
 7   MP       2237 non-null   int64  
 8   FG       2237 non-null   int64  
 9   FGA      2237 non-null   int64  
 10  FG%      2237 non-null   float64
 11  3P       2237 non-null   int64  
 12  3PA      2237 non-null   int64  
 13  3P%      2237 non-null   float64
 14  2P       2237 non-null   int64  
 15  2PA      2237 non-null   int64  
 16  2P%      2237 non-null   float64
 17  eFG%     2237 non-null   float64
 18  FT       2237 non-null   int64  
 19  FTA      2237 non-null   int64  
 20  FT%      2237 non-null   float64
 21  ORB      2237 

In [5]:
stats = df.drop(columns = ["Tm","FG","FGA","FG%","3PA","3P","3P%","2P","2PA","2P%","eFG%","FT","FTA","FT%","ORB","DRB","BLK","PF"])
stats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2237 entries, 0 to 2236
Data columns (total 13 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Player   2237 non-null   object 
 1   Salario  2237 non-null   int64  
 2   Pos      2237 non-null   object 
 3   Age      2237 non-null   int64  
 4   G        2237 non-null   int64  
 5   GS       2237 non-null   int64  
 6   MP       2237 non-null   int64  
 7   TRB      2237 non-null   int64  
 8   AST      2237 non-null   int64  
 9   STL      2237 non-null   int64  
 10  TOV      2237 non-null   int64  
 11  PTS      2237 non-null   int64  
 12  PER      2237 non-null   float64
dtypes: float64(1), int64(10), object(2)
memory usage: 227.3+ KB


In [6]:
stats = stats.sort_values(by=["Player", "Age"], ascending = True)

In [7]:
stats['Temporada'] = stats.groupby('Player').cumcount() + 1

stats.sort_values(by=["Player", "Age"], ascending = True)

,Player,Salario,Pos,Age,G,GS,MP,TRB,AST,STL,TOV,PTS,PER,Temporada
1965,A.J. Green,1801769,ES,24,42,0,401,45,26,3,7,182,21.98,1
1738,AJ Griffin,3536160,AL,19,72,12,1401,153,73,42,42,639,22.01,1
272,Aaron Gordon,19863636,AP,24,62,62,2017,475,228,51,100,894,21.54,1
676,Aaron Gordon,18136364,AP,25,50,50,1384,284,161,33,97,618,21.72,2
913,Aaron Gordon,16409091,AP,26,75,75,2376,439,188,44,133,1126,22.90,3
1612,Aaron Gordon,19690909,AP,27,68,68,2055,446,203,54,98,1109,26.13,4
1961,Aaron Gordon,21266182,AP,28,58,58,1822,375,188,50,85,800,21.33,5
310,Aaron Holiday,2239200,BA,23,66,33,1617,156,225,55,88,627,18.78,1
765,Aaron Holiday,2345640,BA,24,66,8,1176,89,123,46,66,475,19.51,2
914,Aaron Holiday,3980551,BA,25,63,15,1021,122,153,42,67,400,19.00,3


In [8]:
stats.shape

(2237, 14)

In [9]:
stats = stats[stats["Temporada"] <=5]

In [10]:
stats_t = stats.pivot_table(index='Player', columns='Temporada', values=['Salario', 'Pos', 'Age', 'G', 'GS', 'MP', 'TRB', 'AST', 'STL', 'TOV', 'PTS', 'PER'], aggfunc='first')

stats_t.columns = ['{}_{}'.format(col[0], col[1]) for col in stats_t.columns]

In [11]:
stats_t.reset_index(inplace=True)

In [12]:
stats_t

,Player,AST_1,AST_2,AST_3,AST_4,AST_5,Age_1,Age_2,Age_3,Age_4,Age_5,G_1,G_2,G_3,G_4,G_5,GS_1,GS_2,GS_3,GS_4,GS_5,MP_1,MP_2,MP_3,MP_4,MP_5,PER_1,PER_2,PER_3,PER_4,PER_5,PTS_1,PTS_2,PTS_3,PTS_4,PTS_5,Pos_1,Pos_2,Pos_3,Pos_4,Pos_5,STL_1,STL_2,STL_3,STL_4,STL_5,Salario_1,Salario_2,Salario_3,Salario_4,Salario_5,TOV_1,TOV_2,TOV_3,TOV_4,TOV_5,TRB_1,TRB_2,TRB_3,TRB_4,TRB_5
0,A.J. Green,26.00,NaN,NaN,NaN,NaN,24.00,NaN,NaN,NaN,NaN,42.00,NaN,NaN,NaN,NaN,0.00,NaN,NaN,NaN,NaN,401.00,NaN,NaN,NaN,NaN,21.98,NaN,NaN,NaN,NaN,182.00,NaN,NaN,NaN,NaN,ES,NaN,NaN,NaN,NaN,3.00,NaN,NaN,NaN,NaN,1801769.00,NaN,NaN,NaN,NaN,7.00,NaN,NaN,NaN,NaN,45.00,NaN,NaN,NaN,NaN
1,AJ Griffin,73.00,NaN,NaN,NaN,NaN,19.00,NaN,NaN,NaN,NaN,72.00,NaN,NaN,NaN,NaN,12.00,NaN,NaN,NaN,NaN,1401.00,NaN,NaN,NaN,NaN,22.01,NaN,NaN,NaN,NaN,639.00,NaN,NaN,NaN,NaN,AL,NaN,NaN,NaN,NaN,42.00,NaN,NaN,NaN,NaN,3536160.00,NaN,NaN,NaN,NaN,42.00,NaN,NaN,NaN,NaN,153.00,NaN,NaN,NaN,NaN
2,Aaron Gordon,228.00,161.00,188.00,203.00,188.00,24.00,25.00,26.00,27.00,28.00,62.00,50.00,75.00,68.00,58.00,62.00,50.00,75.00,68.00,58.00,2017.00,1384.00,2376.00,2055.00,1822.00,21.54,21.72,22.90,26.13,21.33,894.00,618.00,1126.00,1109.00,800.00,AP,AP,AP,AP,AP,51.00,33.00,44.00,54.00,50.00,19863636.00,18136364.00,16409091.00,19690909.00,21266182.00,100.00,97.00,133.00,98.00,85.00,475.00,284.00,439.00,446.00,375.00
3,Aaron Holiday,225.00,123.00,153.00,89.00,108.00,23.00,24.00,25.00,26.00,27.00,66.00,66.00,63.00,63.00,60.00,33.00,8.00,15.00,6.00,1.00,1617.00,1176.00,1021.00,845.00,1001.00,18.78,19.51,19.00,14.19,19.44,627.00,475.00,400.00,247.00,402.00,BA,BA,BA,BA,BA,55.00,46.00,42.00,37.00,32.00,2239200.00,2345640.00,3980551.00,1836090.00,2346614.00,88.00,66.00,67.00,36.00,45.00,156.00,89.00,122.00,74.00,100.00
4,Aaron Nesmith,23.00,22.00,98.00,80.00,NaN,21.00,22.00,23.00,24.00,NaN,46.00,52.00,73.00,55.00,NaN,1.00,3.00,60.00,30.00,NaN,669.00,574.00,1816.00,1481.00,NaN,15.88,16.56,19.65,21.94,NaN,218.00,196.00,738.00,671.00,NaN,AL,AL,AL,AL,NaN,15.00,20.00,55.00,55.00,NaN,3458400.00,3631200.00,3804360.00,5634257.00,NaN,23.00,31.00,75.00,52.00,NaN,127.00,89.00,277.00,207.00,NaN
5,Aaron Wiggins,68.00,80.00,52.00,NaN,NaN,23.00,24.00,25.00,NaN,NaN,50.00,70.00,60.00,NaN,NaN,35.00,14.00,0.00,NaN,NaN,1209.00,1297.00,878.00,NaN,NaN,16.70,17.88,20.58,NaN,NaN,416.00,479.00,373.00,NaN,NaN,ES,ES,ES,NaN,NaN,30.00,39.00,34.00,NaN,NaN,1000000.00,1563518.00,1836096.00,NaN,NaN,54.00,55.00,36.00,NaN,NaN,178.00,210.00,141.00,NaN,NaN
6,Abdel Nader,38.00,19.00,7.00,NaN,NaN,26.00,27.00,28.00,NaN,NaN,55.00,24.00,14.00,NaN,NaN,6.00,0.00,0.00,NaN,NaN,867.00,355.00,145.00,NaN,NaN,19.24,22.09,12.08,NaN,NaN,345.00,160.00,34.00,NaN,NaN,AL,AL,AL,NaN,NaN,23.00,10.00,8.00,NaN,NaN,1618520.00,1752950.00,2000000.00,NaN,NaN,43.00,19.00,11.00,NaN,NaN,100.00,62.00,27.00,NaN,NaN
7,Admiral Schofield,15.00,25.00,NaN,NaN,NaN,22.00,24.00,NaN,NaN,NaN,33.00,38.00,NaN,NaN,NaN,2.00,1.00,NaN,NaN,NaN,368.00,469.00,NaN,NaN,NaN,13.18,15.22,NaN,NaN,NaN,99.00,146.00,NaN,NaN,NaN,AP,AL,NaN,NaN,NaN,8.00,4.00,NaN,NaN,NaN,1000000.00,300000.00,NaN,NaN,NaN,7.00,21.00,NaN,NaN,NaN,47.00,89.00,NaN,NaN,NaN
8,Al Horford,270.00,94.00,232.00,189.00,140.00,33.00,34.00,35.00,36.00,37.00,67.00,28.00,69.00,63.00,53.00,61.00,28.00,69.00,63.00,24.00,2025.00,782.00,2005.00,1922.00,1413.00,19.18,25.10,17.07,15.63,14.41,798.00,398.00,701.00,616.00,414.00,P,P,P,P,P,52.00,25.00,49.00,30.00,29.00,28000000.00,27500000.00,27000000.00,26500000.00,10000000.00,80.00,29.00,65.00,37.00,37.00,456.00,188.00,530.00,390.00,352.00
9,Al-Farouq Aminu,21.00,31.00,NaN,NaN,NaN,29.00,30.00,NaN,NaN,NaN,18.00,23.00,NaN,NaN,NaN,2.00,14.00,NaN,NaN,NaN,380.00,434.00,NaN,NaN,NaN,10.67,11.96,NaN,NaN,NaN,78.00,102.00,NaN,NaN,NaN,AP,AP,NaN,NaN,NaN,18.00,19.00,NaN,NaN,NaN,9258000.00,9720900.00,NaN,NaN,NaN,17.00,28.00,NaN,NaN,NaN,87.00,110.00,NaN,NaN,NaN


In [13]:
stats_t["Age_2"] = stats_t["Age_1"].transform(lambda x: x.ffill()+1)

stats_t["Age_3"] = stats_t["Age_2"].transform(lambda x: x.ffill()+1)

stats_t["Age_4"] = stats_t["Age_2"].transform(lambda x: x.ffill()+1)

stats_t["Age_5"] = stats_t["Age_2"].transform(lambda x: x.ffill()+1)


stats_t["AST_2"] = stats_t["AST_1"].transform(lambda x: x.ffill()+1)

stats_t["AST_3"] = stats_t["AST_1"].transform(lambda x: x.ffill())

stats_t["AST_4"] = stats_t["AST_1"].transform(lambda x: x.ffill())

stats_t["AST_5"] = stats_t["AST_1"].transform(lambda x: x.ffill())


stats_t["G_2"] = stats_t["G_1"].transform(lambda x: x.ffill())

stats_t["G_3"] = stats_t["G_1"].transform(lambda x: x.ffill())

stats_t["G_4"] = stats_t["G_1"].transform(lambda x: x.ffill())

stats_t["G_5"] = stats_t["G_1"].transform(lambda x: x.ffill())

stats_t["GS_2"] = stats_t["GS_1"].transform(lambda x: x.ffill())

stats_t["GS_3"] = stats_t["GS_1"].transform(lambda x: x.ffill())

stats_t["GS_4"] = stats_t["GS_1"].transform(lambda x: x.ffill())

stats_t["GS_5"] = stats_t["GS_1"].transform(lambda x: x.ffill())

stats_t["MP_2"] = stats_t["MP_1"].transform(lambda x: x.ffill())

stats_t["MP_3"] = stats_t["MP_1"].transform(lambda x: x.ffill())

stats_t["MP_4"] = stats_t["MP_1"].transform(lambda x: x.ffill())

stats_t["MP_5"] = stats_t["MP_1"].transform(lambda x: x.ffill())

stats_t["PER_2"] = stats_t["PER_1"].transform(lambda x: x.ffill())

stats_t["PER_3"] = stats_t["PER_1"].transform(lambda x: x.ffill())

stats_t["PER_4"] = stats_t["PER_1"].transform(lambda x: x.ffill())

stats_t["PER_5"] = stats_t["PER_1"].transform(lambda x: x.ffill())

stats_t["PTS_2"] = stats_t["PTS_1"].transform(lambda x: x.ffill())

stats_t["PTS_3"] = stats_t["PTS_1"].transform(lambda x: x.ffill())

stats_t["PTS_4"] = stats_t["PTS_1"].transform(lambda x: x.ffill())

stats_t["PTS_5"] = stats_t["PTS_1"].transform(lambda x: x.ffill())

stats_t["STL_2"] = stats_t["STL_1"].transform(lambda x: x.ffill())

stats_t["STL_3"] = stats_t["STL_1"].transform(lambda x: x.ffill())

stats_t["STL_4"] = stats_t["STL_1"].transform(lambda x: x.ffill())

stats_t["STL_5"] = stats_t["STL_1"].transform(lambda x: x.ffill())

stats_t["Salario_2"] = stats_t["Salario_1"].transform(lambda x: x.ffill())

stats_t["Salario_3"] = stats_t["Salario_1"].transform(lambda x: x.ffill())

stats_t["Salario_4"] = stats_t["Salario_1"].transform(lambda x: x.ffill())

stats_t["Salario_5"] = stats_t["Salario_1"].transform(lambda x: x.ffill())

stats_t["TOV_2"] = stats_t["TOV_1"].transform(lambda x: x.ffill())

stats_t["TOV_3"] = stats_t["TOV_1"].transform(lambda x: x.ffill())

stats_t["TOV_4"] = stats_t["TOV_1"].transform(lambda x: x.ffill())

stats_t["TOV_5"] = stats_t["TOV_1"].transform(lambda x: x.ffill())

stats_t["TRB_2"] = stats_t["TRB_1"].transform(lambda x: x.ffill())

stats_t["TRB_3"] = stats_t["TRB_1"].transform(lambda x: x.ffill())

stats_t["TRB_4"] = stats_t["TRB_1"].transform(lambda x: x.ffill())

stats_t["TRB_5"] = stats_t["TRB_1"].transform(lambda x: x.ffill())

stats_t["PER_2"] = stats_t["PER_1"].transform(lambda x: x.ffill())

stats_t["PER_3"] = stats_t["PER_1"].transform(lambda x: x.ffill())

stats_t["PER_4"] = stats_t["PER_1"].transform(lambda x: x.ffill())

stats_t["PER_5"] = stats_t["PER_1"].transform(lambda x: x.ffill())


In [14]:
df_numeric = stats_t.select_dtypes(include=np.number)

In [15]:
X = df_numeric.drop(["Salario_5","Salario_4","Salario_3","Salario_2","Salario_1"], axis = 1)
 
y = df_numeric.Salario_5

In [16]:
X_train, X_test, y_train, y_test = tts(X, y,
                                       train_size=0.8,
                                       test_size=0.2,
                                       random_state=42
                                      )

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((618, 50), (155, 50), (618,), (155,))

In [17]:
from lazypredict.Supervised import LazyRegressor

#Split the data into training and testing sets (if not already split)

# Initialize Lazy Regressor
reg = LazyRegressor(verbose=0, ignore_warnings=True, custom_metric=None)
# Fit and score all models
models, predictions = reg.fit(X_train, X_test, y_train, y_test)

print(models)

100%|██████████████████████████████████████████████████████████████████████████████████| 42/42 [00:10<00:00,  3.96it/s]

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000267 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6395
[LightGBM] [Info] Number of data points in the train set: 618, number of used features: 50
[LightGBM] [Info] Start training from score 5653667.071197
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive g

In [18]:
from sklearn.ensemble import RandomForestRegressor as rfr
rfr = rfr(random_state=42)

rfr.fit(X_train, y_train)

y_pred = rfr.predict(X_test)

In [19]:
rfr

RandomForestRegressor(random_state=42)

In [20]:
pd.set_option('display.float_format', '{:.2f}'.format)
y_pred = pd.DataFrame(y_pred)

rmse = mean_squared_error(y_test, y_pred, squared=False)


a = pd.DataFrame(y_test)

a = a.reset_index(drop = True)

y_pred["real"]= a

y_pred["error"]= y_pred[0] - y_pred["real"]

y_pred

,0,real,error
0,8809070.30,19000000.00,-10190929.70
1,1079338.10,934168.00,145170.10
2,12305423.86,25595700.00,-13290276.14
3,10601723.08,1620564.00,8981159.08
4,3254748.74,5170564.00,-1915815.26
5,16069153.42,13437500.00,2631653.42
6,2166763.38,1737145.00,429618.38
7,1475446.83,1445697.00,29749.83
8,255874.72,898310.00,-642435.28
9,1954791.51,707770.00,1247021.51


In [21]:
rmse

5533988.979504351

In [ ]:
import joblib
joblib.dump(rfr, 'modelo_random_forest.joblib')